In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 176ms/step - loss: 0.6876 - accuracy: 0.5663
Epoch 2/30
58/58 [==============================] - 10s 173ms/step - loss: 0.5825 - accuracy: 0.6944
Epoch 3/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5095 - accuracy: 0.7371
Epoch 4/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4471 - accuracy: 0.7886
Epoch 5/30
58/58 [==============================] - 10s 177ms/step - loss: 0.3845 - accuracy: 0.8412
Epoch 6/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2982 - accuracy: 0.8795
Epoch 7/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2244 - accuracy: 0.9157
Epoch 8/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2126 - accuracy: 0.9036
Epoch 9/30
58/58 [==============================] - 10s 178ms/step - loss: 0.1533 - accuracy: 0.9354
Epoch 10/30
58/58 [==============================] - 10s 176ms/step - loss: 0.1232 - accura

58/58 [==============================] - 11s 182ms/step - loss: 0.0787 - accuracy: 0.9737
Epoch 22/30
58/58 [==============================] - 11s 183ms/step - loss: 0.0682 - accuracy: 0.9759
Epoch 23/30
58/58 [==============================] - 11s 182ms/step - loss: 0.0663 - accuracy: 0.9781
Epoch 24/30
58/58 [==============================] - 11s 181ms/step - loss: 0.0797 - accuracy: 0.9671
Epoch 25/30
58/58 [==============================] - 11s 181ms/step - loss: 0.0823 - accuracy: 0.9671
Epoch 26/30
58/58 [==============================] - 10s 181ms/step - loss: 0.0530 - accuracy: 0.9836
Epoch 27/30
58/58 [==============================] - 10s 180ms/step - loss: 0.0720 - accuracy: 0.9737
Epoch 28/30
58/58 [==============================] - 11s 181ms/step - loss: 0.0734 - accuracy: 0.9704
Epoch 29/30
58/58 [==============================] - 11s 181ms/step - loss: 0.0505 - accuracy: 0.9803
Epoch 30/30
58/58 [==============================] - 10s 170ms/step - loss: 0.0439 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 17s 297ms/step - loss: 0.6251 - accuracy: 0.6451
Epoch 2/30
58/58 [==============================] - 17s 297ms/step - loss: 0.4527 - accuracy: 0.8061
Epoch 3/30
58/58 [==============================] - 17s 297ms/step - loss: 0.3373 - accuracy: 0.8664
Epoch 4/30
58/58 [==============================] - 17s 297ms/step - loss: 0.2479 - accuracy: 0.9080
Epoch 5/30
58/58 [==============================] - 17s 296ms/step - loss: 0.1452 - accuracy: 0.9540
Epoch 6/30
58/58 [==============================] - 17s 298ms/step - loss: 0.2241 - accuracy: 0.9310
Epoch 7/30
58/58 [==============================] - 17s 295ms/step - loss: 0.1377 - accuracy: 0.9562
Epoch 8/30
58/58 [==============================] - 17s 297ms/step - loss: 0.0793 - accuracy: 0.9781
Epoch 9/30
58/58 [==============================] - 17s 296ms/step - loss: 0.0708 - accuracy: 0.9759
Epoch 10/30
58/58 [==============================] - 17s 298ms/step - loss: 0.0416 - accura

58/58 [==============================] - 19s 323ms/step - loss: 0.0036 - accuracy: 0.9989
Epoch 22/30
58/58 [==============================] - 19s 322ms/step - loss: 0.0032 - accuracy: 0.9989
Epoch 23/30
58/58 [==============================] - 19s 322ms/step - loss: 0.0029 - accuracy: 0.9978
Epoch 24/30
58/58 [==============================] - 19s 326ms/step - loss: 0.0029 - accuracy: 0.9989
Epoch 25/30
58/58 [==============================] - 19s 324ms/step - loss: 0.0021 - accuracy: 0.9989
Epoch 26/30
58/58 [==============================] - 19s 323ms/step - loss: 0.0031 - accuracy: 0.9978
Epoch 27/30
58/58 [==============================] - 19s 324ms/step - loss: 0.0028 - accuracy: 0.9989
Epoch 28/30
58/58 [==============================] - 19s 323ms/step - loss: 0.0023 - accuracy: 0.9989
Epoch 29/30
58/58 [==============================] - 19s 323ms/step - loss: 0.0020 - accuracy: 0.9978
Epoch 30/30
58/58 [==============================] - 19s 323ms/step - loss: 0.0018 - accuracy: